ESOL data

1. No lost through standartization - 1128 molecules
2. 5 fully duplicated rows have been removed -> 1123 rows
3. After removing strange molecules 1064 molecules have left
4. Among 6 duplicated smiles the most commong logS left -> final size = 1058 entries


FreeSolv data

1. No lost through standartization - 642 molecules
2. No duplicated rows/values have been obtained
3. After removing strange molecules 565 molecules have left


In [68]:
from pathlib import Path
import pandas as pd
import os
import numpy as np

In [69]:
import sys
sys.path
sys.path.append('../scripts/preprocess')
from standardize_smiles import StandardizeTautomers

In [70]:
INPUT_PATH = Path("../data/1_filtering/esol_freesolv")

OUTPUT_PATH = Path("../data/2_standardize/esol_freesolv")

In [71]:
StandardizeTautomers().standardize(INPUT_PATH, OUTPUT_PATH)

2020-09-21 11:23:32.923 | INFO     | standardize_smiles:standardize:38 - Processing freesolv.csv
100%|██████████| 642/642 [00:01<00:00, 598.09it/s]
2020-09-21 11:23:34.105 | INFO     | standardize_smiles:standardize:38 - Processing esol.csv
100%|██████████| 1128/1128 [00:07<00:00, 150.71it/s]


In [72]:
with os.scandir(INPUT_PATH) as entries:
    initial_datasets = dict([(entry.name, pd.read_csv(entry, index_col=0)) \
                     for entry in entries if entry.is_file()])

In [73]:
with os.scandir(OUTPUT_PATH) as entries:
    standardized_datasets = dict([(entry.name, pd.read_csv(entry, index_col=0)) \
                     for entry in entries if entry.is_file() and '.csv' in entry.name])

In [74]:
datasets_stats = pd.DataFrame([{'Dataset name' : dataset_name, \
  'Initial size' : initial_datasets[dataset_name].shape[0], \
  'Standardized size' : standardized_datasets[dataset_name].shape[0], \
  '% of loss' : round((initial_datasets[dataset_name].shape[0]-standardized_datasets[dataset_name].shape[0]) \
                                /standardized_datasets[dataset_name].shape[0]*100,1)} \
 for dataset_name in initial_datasets.keys()] + \
[{'Dataset name' : 'Total amount', \
  'Initial size' : np.sum([dataset.shape[0] for data_name, dataset in initial_datasets.items()]), \
  'Standardized size' : np.sum([dataset.shape[0] for data_name, dataset in standardized_datasets.items()]), \
  '% of loss' : round((np.sum([dataset.shape[0] for data_name, dataset in initial_datasets.items()])- \
                       np.sum([dataset.shape[0] for data_name, dataset in standardized_datasets.items()])) \
                                /np.sum([dataset.shape[0] for data_name, dataset in standardized_datasets.items()])*100,1)}])

In [75]:
datasets_stats

,Dataset name,Initial size,Standardized size,% of loss
0,freesolv.csv,642,642,0.0
1,esol.csv,1128,1128,0.0
2,Total amount,1770,1770,0.0


# Check duplicates

## Esol

In [76]:
DATA_PATH = Path("../data/2_standardize/esol_freesolv")

FILENAME = 'esol.csv'

In [77]:
SMILES_COLUMN = 'smiles'
VALUE_COLUMN = 'logS'

In [78]:
esol_data = pd.read_csv(os.path.join(DATA_PATH, FILENAME))

In [79]:
esol_data.head()

,smiles,logS
0,N#CC(OC1OC(COC2OC(CO)C(O)C(O)C2O)C(O)C(O)C1O)c...,-0.77
1,Cc1occc1C(=O)Nc1ccccc1,-3.30
2,CC(C)=CCCC(C)=CC=O,-2.06
3,c1ccc2c(c1)ccc1c2ccc2c3ccccc3ccc21,-7.87
4,c1ccsc1,-1.33


In [80]:
esol_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1128 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  1128 non-null   object 
 1   logS    1128 non-null   float64
dtypes: float64(1), object(1)
memory usage: 17.8+ KB


In [81]:
esol_data_wo_dup = esol_data.drop_duplicates()

In [82]:
esol_data_wo_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1123 entries, 0 to 1127
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  1123 non-null   object 
 1   logS    1123 non-null   float64
dtypes: float64(1), object(1)
memory usage: 26.3+ KB


In [83]:
esol_data_wo_dup[SMILES_COLUMN].duplicated().sum()

6

In [84]:
PROCESSED_PATH = "../data/3_final_data"

esol_data_wo_dup.to_csv(os.path.join(PROCESSED_PATH, 'esol.csv'), index = False)

## FreeSolv

In [99]:
DATA_PATH = Path("../data/2_standardize/esol_freesolv")

FILENAME = 'freesolv.csv'

In [100]:
SMILES_COLUMN = 'smiles'
VALUE_COLUMN = 'Energy'

In [101]:
freesolv_data = pd.read_csv(os.path.join(DATA_PATH, FILENAME))

In [102]:
freesolv_data.head()

,smiles,Energy
0,COc1ccc(C(=O)N(C)C)cc1,-11.01
1,CS(=O)(=O)Cl,-4.87
2,C=CC(C)C,1.83
3,CCc1cnccn1,-5.45
4,CCCCCCCO,-4.21


In [103]:
freesolv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 642 entries, 0 to 641
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  642 non-null    object 
 1   Energy  642 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.2+ KB


In [104]:
freesolv_data_wo_dup = freesolv_data.drop_duplicates()

In [105]:
freesolv_data_wo_dup.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 642 entries, 0 to 641
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  642 non-null    object 
 1   Energy  642 non-null    float64
dtypes: float64(1), object(1)
memory usage: 15.0+ KB


In [106]:
freesolv_data_wo_dup[SMILES_COLUMN].duplicated().sum()

0

In [107]:
PROCESSED_PATH = "../data/3_final_data"

freesolv_data_wo_dup.to_csv(os.path.join(PROCESSED_PATH, 'freesolv.csv'), index = False)

# Remove strange mols 

In [85]:
from remove_strange_mols import remove_strange_mols

## ESOL

In [86]:
DATA_PATH = "../data/3_final_data"
FILENAME = 'esol.csv'

In [87]:
remove_strange_mols(os.path.join(DATA_PATH, FILENAME), os.path.join(DATA_PATH, FILENAME))

2020-09-21 11:23:46.831 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule N#CC(OC1OC(COC2OC(CO)C(O)C(O)C2O)C(O)C(O)C1O)c1ccccc1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.832 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Cc1occc1C(=O)Nc1ccccc1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.833 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CC(C)=CCCC(C)=CC=O isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.834 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule c1ccc2c(c1)ccc1c2ccc2c3ccccc3ccc21 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.835 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule c1ccsc1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.836 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule c1ccc2sc

2020-09-21 11:23:46.884 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule COC(=O)C1=C(C)N=C(C)C(C(=O)OC)C1c1ccccc1[N+](=O)[O-] isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.885 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Cc1ccc2ccc(C)nc2c1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.885 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule C#CCCCCCC isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.886 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCC1(C2=CCCCC2)C(=O)NC(=O)NC1=O isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.887 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule c1ccc2c(c1)ccc1c3ccccc3ccc21 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.888 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCC(

2020-09-21 11:23:46.937 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CC=C(c1ccc(O)cc1)C(CC)c1ccc(O)cc1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.938 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule N#Cc1c(Cl)c(Cl)c(Cl)c(C#N)c1Cl isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.939 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Clc1ccc(Cl)c(-c2ccc(Cl)c(Cl)c2)c1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.940 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule c1ccc(C2CO2)cc1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.941 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CC(C)c1ccccc1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.942 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CC12CCC(=O)C=C1CCC

2020-09-21 11:23:46.989 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Nc1cccc2ccccc12 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.990 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Cc1cccc(C)c1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.990 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule O=c1nc2nccnc2c[nH]1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.991 | DEBUG    | remove_strange_mols:check_not_strange_mols:40 - The molecule CO is in the list of very soluble in water molecules
2020-09-21 11:23:46.992 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCC1(CCC(C)C)C(=O)NC(=O)NC1=O isn't in the list of very soluble in water molecules
2020-09-21 11:23:46.993 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCC(C)=O isn't in the list of very soluble in water molecules
2020-0

2020-09-21 11:23:47.042 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule COCC(=O)N(c1c(C)cccc1C)C(C)C(=O)OC isn't in the list of very soluble in water molecules
2020-09-21 11:23:47.043 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CNC(=O)Oc1ccccc1OC(C)C isn't in the list of very soluble in water molecules
2020-09-21 11:23:47.044 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCC(C)Cl isn't in the list of very soluble in water molecules
2020-09-21 11:23:47.045 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Oc1ccc2ccccc2c1 isn't in the list of very soluble in water molecules
2020-09-21 11:23:47.046 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CC(C)Oc1cc(-n2nc(C(C)(C)C)oc2=O)c(Cl)cc1Cl isn't in the list of very soluble in water molecules
2020-09-21 11:23:47.048 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule C#CCCCC isn't in the 

...

2020-09-21 11:23:51.110 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCC has more than 5 atoms
2020-09-21 11:23:51.110 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule COC(=O)c1cccnc1 has more than 5 atoms
2020-09-21 11:23:51.111 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule NS(=O)(=O)c1cc2c(cc1C(F)(F)F)NC(Cc1ccccc1)NS2(=O)=O has more than 5 atoms
2020-09-21 11:23:51.112 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Clc1ccc(-c2cc(Cl)c(Cl)c(Cl)c2Cl)cc1Cl has more than 5 atoms
2020-09-21 11:23:51.114 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C=CC1(C)OC(=O)N(c2cc(Cl)cc(Cl)c2)C1=O has more than 5 atoms
2020-09-21 11:23:51.115 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCN=c1[nH]c(Cl)nc(=NC(C)(C)C#N)[nH]1 has more than 5 atoms
2020-09-21 11:23:51.117 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule c1ccc2c(c1)c1ccccc1c1ccccc21 has more than 5 atoms
2020-09-21 11:23:51.

2020-09-21 11:23:51.176 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CC12CCC3C(CCC4CC(=O)CCC43C)C1CCC2O has more than 5 atoms
2020-09-21 11:23:51.177 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C=CC(O)CCC has more than 5 atoms
2020-09-21 11:23:51.178 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Oc1[nH]c(O)c2c1C1C=C(C(O)(c3ccccc3)c3ccccn3)C2C1=C(c1ccccc1)c1ccccn1 has more than 5 atoms
2020-09-21 11:23:51.179 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCOCCCC has more than 5 atoms
2020-09-21 11:23:51.180 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCCCCCCCCO has more than 5 atoms
2020-09-21 11:23:51.181 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCN1c2nc(N(C)CCO)ccc2NC(=O)c2cccnc21 has more than 5 atoms
2020-09-21 11:23:51.181 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCC(C)(C)O has more than 5 atoms
2020-09-21 11:23:51.182 | DEBUG    | remove_stran

2020-09-21 11:23:51.246 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCN(CCC)c1c([N+](=O)[O-])cc(C(F)(F)F)cc1[N+](=O)[O-] has more than 5 atoms
2020-09-21 11:23:51.247 | DEBUG    | remove_strange_mols:check_atoms:71 - The molecule CCO has less than 5 atoms
2020-09-21 11:23:51.248 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule O=C1NC(=O)C2(CCCC2)C(=O)N1 has more than 5 atoms
2020-09-21 11:23:51.249 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCNC(=O)C(C)OC(=O)Nc1ccccc1 has more than 5 atoms
2020-09-21 11:23:51.250 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CC(C)=CC1C(C(=O)OCc2cccc(Oc3ccccc3)c2)C1(C)C has more than 5 atoms
2020-09-21 11:23:51.251 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CN(C)C(=O)NC1CCCCCCC1 has more than 5 atoms
2020-09-21 11:23:51.252 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule ClC1(Cl)C2(Cl)C3(Cl)C4(Cl)C(Cl)(Cl)C5(Cl)C3(Cl)C1(Cl)C5(Cl)C24Cl has more than 

2020-09-21 11:23:51.313 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Sc1ccccc1 has more than 5 atoms
2020-09-21 11:23:51.314 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CNC(=O)Oc1cc(C)cc(C)c1 has more than 5 atoms
2020-09-21 11:23:51.315 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule ClC1=C(Cl)C2(Cl)C3C(Cl)C(Cl)CC3C1(Cl)C2(Cl)Cl has more than 5 atoms
2020-09-21 11:23:51.316 | DEBUG    | remove_strange_mols:check_atoms:71 - The molecule CSSC has less than 5 atoms
2020-09-21 11:23:51.317 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule NC(=O)c1ccccc1 has more than 5 atoms
2020-09-21 11:23:51.318 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Clc1ccccc1Br has more than 5 atoms
2020-09-21 11:23:51.319 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule COC(=O)c1ccccc1OC1OC(COC2OCC(O)C(O)C2O)C(O)C(O)C1O has more than 5 atoms
2020-09-21 11:23:51.320 | DEBUG    | remove_strange_mols:check_atoms:73 - 

2020-09-21 11:23:51.381 | DEBUG    | remove_strange_mols:check_atoms:71 - The molecule C=CC has less than 5 atoms
2020-09-21 11:23:51.382 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Oc1ccc(Cl)cc1Cc1cc(Cl)ccc1O has more than 5 atoms
2020-09-21 11:23:51.383 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCOC(=O)Nc1cccc(OC(=O)Nc2ccccc2)c1 has more than 5 atoms
2020-09-21 11:23:51.384 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule O=C1c2ccccc2C(=O)c2ccccc21 has more than 5 atoms
2020-09-21 11:23:51.385 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCCC(C)O has more than 5 atoms
2020-09-21 11:23:51.386 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CC1=C(C(=O)Nc2ccccc2)S(=O)(=O)CCO1 has more than 5 atoms
2020-09-21 11:23:51.388 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCc1ccccc1 has more than 5 atoms
2020-09-21 11:23:51.389 | DEBUG    | remove_strange_mols:check_atoms:73 - The mole

In [88]:
esol_data = pd.read_csv(os.path.join(DATA_PATH, FILENAME))

In [89]:
esol_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1064 entries, 0 to 1063
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  1064 non-null   object 
 1   logS    1064 non-null   float64
dtypes: float64(1), object(1)
memory usage: 16.8+ KB


In [91]:
esol_data.duplicated().sum()

0

In [90]:
esol_data[SMILES_COLUMN].duplicated().sum()

6

In [92]:
esol_data_wo_aver = esol_data.groupby([SMILES_COLUMN]).agg(lambda x:x.value_counts().index[0])

In [93]:
esol_data_wo_aver.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1058 entries, BrC(Br)(Br)Br to c1cncnc1
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   logS    1058 non-null   float64
dtypes: float64(1)
memory usage: 16.5+ KB


In [94]:
PROCESSED_PATH = "../data/3_final_data"
esol_data_wo_aver.to_csv(os.path.join(PROCESSED_PATH, 'esol.csv'))

## FreeSolv

In [109]:
DATA_PATH = "../data/3_final_data"
FILENAME = 'freesolv.csv'

In [110]:
remove_strange_mols(os.path.join(DATA_PATH, FILENAME), os.path.join(DATA_PATH, FILENAME))

2020-09-21 11:37:39.474 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule COc1ccc(C(=O)N(C)C)cc1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.476 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CS(=O)(=O)Cl isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.477 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule C=CC(C)C isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.479 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCc1cnccn1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.480 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCCCCCCO isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.482 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Cc1cc(C)cc(O)c1 isn't in the list of very soluble in water molecules
2020-09-21 11

2020-09-21 11:37:39.542 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule FC(F)(F)F isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.544 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CC[C@@H](C)CO isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.545 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Oc1ccccc1I isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.546 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule COc1cccc(OC)c1O isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.547 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule C#CCC isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.548 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule FC(F)(F)c1ccccc1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.

2020-09-21 11:37:39.607 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCCCl isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.608 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCCSCCC isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.609 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCC[C@@H](O)CC isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.610 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CC#N isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.612 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CN(CC(F)(F)F)c1ccccc1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.613 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule FC(F)O[C@H](Cl)C(F)(F)F isn't in the list of very soluble in water molecules
2020-09-21 11:

2020-09-21 11:37:39.677 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Cc1ccc(C(C)C)cc1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.678 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Cn1ccnc1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.679 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule OCCO isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.680 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Clc1ccccc1Cl isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.684 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule O=c1cc(Cl)[nH]c(=O)[nH]1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.685 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCCOC=O isn't in the list of very soluble in water molecules
2020-09-21 11:37:

2020-09-21 11:37:39.740 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCCCCCOC(C)=O isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.741 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule O=C1CCCC1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.742 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCCCC(=O)O isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.743 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCBr isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.744 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule Cc1ccc2cc(C)ccc2c1 isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.753 | DEBUG    | remove_strange_mols:check_not_strange_mols:44 - The molecule CCCCCCO isn't in the list of very soluble in water molecules
2020-09-21 11:37:39.754 | D

...

2020-09-21 11:37:41.830 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Cc1ccccc1N has more than 5 atoms
2020-09-21 11:37:41.831 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCBr has more than 5 atoms
2020-09-21 11:37:41.832 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCCCCCO has more than 5 atoms
2020-09-21 11:37:41.833 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Cc1ccncc1 has more than 5 atoms
2020-09-21 11:37:41.836 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule ClC(Cl)=C(Cl)Cl has more than 5 atoms
2020-09-21 11:37:41.837 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CC(C)(C)Br has more than 5 atoms
2020-09-21 11:37:41.838 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C=C(c1ccccc1)c1ccccc1 has more than 5 atoms
2020-09-21 11:37:41.838 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCc1ccc(C)cc1 has more than 5 atoms
2020-09-21 11:37:41.839 | 

2020-09-21 11:37:41.898 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Clc1cc2c(cc1Cl)Oc1cc(Cl)c(Cl)cc1O2 has more than 5 atoms
2020-09-21 11:37:41.899 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule N#Cc1c(Cl)cccc1Cl has more than 5 atoms
2020-09-21 11:37:41.902 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCOC=O has more than 5 atoms
2020-09-21 11:37:41.903 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Clc1cc(Cl)c(Cl)cc1Cl has more than 5 atoms
2020-09-21 11:37:41.904 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCOC(OCC)Oc1ccccc1 has more than 5 atoms
2020-09-21 11:37:41.905 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule O=[N+]([O-])c1cccc(O)c1 has more than 5 atoms
2020-09-21 11:37:41.906 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCCCCO has more than 5 atoms
2020-09-21 11:37:41.907 | DEBUG    | remove_strange_mols:check_atoms:71 - The molecule C=CCC has less t

2020-09-21 11:37:41.968 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C[C@@H](CCO[N+](=O)[O-])O[N+](=O)[O-] has more than 5 atoms
2020-09-21 11:37:41.969 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CC(C)OC(C)C has more than 5 atoms
2020-09-21 11:37:41.972 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCC(C)C has more than 5 atoms
2020-09-21 11:37:41.973 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule O=[N+]([O-])c1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl has more than 5 atoms
2020-09-21 11:37:41.974 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule FC(F)(F)[C@H](Cl)Br has more than 5 atoms
2020-09-21 11:37:41.975 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCOCCCC has more than 5 atoms
2020-09-21 11:37:41.976 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCC1CCCC1 has more than 5 atoms
2020-09-21 11:37:41.977 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CC(C)CC(

2020-09-21 11:37:42.039 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CC(C)(C)c1ccccc1 has more than 5 atoms
2020-09-21 11:37:42.040 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCC(=O)OC has more than 5 atoms
2020-09-21 11:37:42.041 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C[C@H](O)C(F)(F)F has more than 5 atoms
2020-09-21 11:37:42.042 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCBr has more than 5 atoms
2020-09-21 11:37:42.042 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C=CCCCCCC has more than 5 atoms
2020-09-21 11:37:42.043 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CC1=CC(=O)C(C(C)C)CC1 has more than 5 atoms
2020-09-21 11:37:42.047 | DEBUG    | remove_strange_mols:check_atoms:71 - The molecule CC(C)O has less than 5 atoms
2020-09-21 11:37:42.048 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCCCN has more than 5 atoms
2020-09-21 11:37:42.048 |

2020-09-21 11:37:42.111 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C1=CCCC1 has more than 5 atoms
2020-09-21 11:37:42.112 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C[C@@H](O)C(F)(F)F has more than 5 atoms
2020-09-21 11:37:42.113 | DEBUG    | remove_strange_mols:check_atoms:71 - The molecule CCCN has less than 5 atoms
2020-09-21 11:37:42.114 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule O=[N+]([O-])c1ccccc1O has more than 5 atoms
2020-09-21 11:37:42.115 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Cc1cccc2ccccc12 has more than 5 atoms
2020-09-21 11:37:42.116 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule Clc1c(Cl)c(Cl)c(Cl)c(Cl)c1Cl has more than 5 atoms
2020-09-21 11:37:42.118 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule CCCCC/C=C/C=O has more than 5 atoms
2020-09-21 11:37:42.120 | DEBUG    | remove_strange_mols:check_atoms:73 - The molecule C#CCCCCCC has more than 5 atoms
2020

In [111]:
freesolv_data = pd.read_csv(os.path.join(DATA_PATH, FILENAME))

In [112]:
freesolv_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   smiles  565 non-null    object 
 1   Energy  565 non-null    float64
dtypes: float64(1), object(1)
memory usage: 9.0+ KB


In [113]:
freesolv_data.duplicated().sum()

0

In [114]:
freesolv_data[SMILES_COLUMN].duplicated().sum()

0

# Split_data

In [116]:
from sklearn.model_selection import train_test_split
import pandas as pd

DATA_PATH = "../data/3_final_data/"
data_names = ['esol', 'freesolv']

In [117]:
def train_test_validation_split(df):
    train_data, rest_data = train_test_split(df, test_size=0.3)
    test_data, validation_data = train_test_split(rest_data, test_size=0.5)
    return train_data.reset_index(drop=True), validation_data.reset_index(drop=True), test_data.reset_index(drop=True)

In [119]:
for file in data_names:
    
    data = pd.read_csv(DATA_PATH + file + '.csv')
    
    
    print(file, 'shape: ', data.shape)    
    train, validation, test = train_test_validation_split(data)
    print('SPLITTED SHAPES:\n\ttrain: {0}\n\tvalidation: {1}\n\ttest: {2}\n'.format(train.shape, validation.shape, test.shape))
    
    train.to_csv(DATA_PATH + 'split_data/' + file + '_train.csv')
    validation.to_csv(DATA_PATH + 'split_data/' + file + '_validation.csv')
    test.to_csv(DATA_PATH + 'split_data/' + file + '_test.csv')

esol shape:  (1058, 2)
SPLITTED SHAPES:
	train: (740, 2)
	validation: (159, 2)
	test: (159, 2)

freesolv shape:  (565, 2)
SPLITTED SHAPES:
	train: (395, 2)
	validation: (85, 2)
	test: (85, 2)

